## Data Preparation

You should prepare the following things before running this step. 

1. **NIfTI images** of fixed CT
   - we are going to use it to simulate the noise-free ground truth and noisy thin-slice counterpart.

2. **A patient list** that emunarates these fixed CT.  
   - we have 100 fixed CT in this study.


---

## Task: Simulate noise-free ground truth and noisy thin-slice counterpart

- In this script, we start with any fixed CT scan. We first resample it to 5mm using averaging.

- Then we resample this ground truth into 0.625mm using interpolation, which will be considered as **"noise-free thin slice"**. We select middle 100 slices, otherwise the data is too big. The file is named as ```fixedCT/img_thinslice_partial.nii.gz```

- Lastly, we insert noise into thin slices (using two types of noises), which will be considered as **"noisy thin slice"**. The file is named as ```simulation/recon.nii.gz```

---

### Docker environment
Please use `docker/docker_tensorflow`, it will build a tensorflow docker


In [1]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys
import numpy as np 
import pandas as pd
import nibabel as nb
from skimage.measure import block_reduce

import CTDenoising_Diffusion_N2N.functions_collection as ff
import CTDenoising_Diffusion_N2N.Data_processing as Data_processing

data_path = '/mnt/camca_NAS/Portable_CT_data'
main_path = '/mnt/camca_NAS/denoising/'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### step 1: resample all fixed CT to 5mm using averaging, then interpolate to 0.625mm

In [6]:
# read patient list
patient_sheet = pd.read_excel(os.path.join(main_path,'Patient_lists', 'fixedCT_static.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
print('patient sheet len: ', len(patient_sheet))

for i in range(0, len(patient_sheet)):
    row = patient_sheet.iloc[i]
    patient_id = row['Patient_ID']
    patient_subID = row['Patient_subID']
    use = row['use']

    original_file = os.path.join(data_path,'nii_imgs_202404/static',patient_id,patient_subID,'fixed', use+'.nii.gz')
    
    # get the affine and pixel dimension
    img = nb.load(original_file)
    affine = img.affine
    pixdim = img.header.get_zooms()
    img_data = img.get_fdata()

    # define save folder
    save_folder = os.path.join(main_path, 'Data/fixedCT', patient_id, patient_subID)
    ff.make_folder([os.path.join(main_path,'Data/fixedCT'), os.path.join(main_path,'Data/fixedCT', patient_id), os.path.join(main_path,'Data/fixedCT', patient_id, patient_subID)])

    ######### first, resample slice thickness to 5mm using averaging
    z_scale_factor = int(5 // pixdim[2])
    print('z_scale_factor: ', z_scale_factor)
    img_data_xyz5mm = block_reduce(img_data, (1,1,z_scale_factor), np.mean)

    # change affine and pixel dimension accordingly
    new_affine_5mm = affine.copy()
    new_affine_5mm[2, 2] *= z_scale_factor
    new_pixdim_5mm = (pixdim[0],pixdim[1], pixdim[2]*z_scale_factor)
    # save in the header
    img.header.set_zooms(new_pixdim_5mm)

    # save the image
    save_file = os.path.join(save_folder, 'img_5mm.nii.gz')
    nb.save(nb.Nifti1Image(img_data_xyz5mm, new_affine_5mm, img.header), save_file)

    ######### second, resample slice thickness to 0.625mm using interpolation
    new_dim = [pixdim[0], pixdim[1], 0.625]

    img_5mm = nb.load(os.path.join(save_folder, 'img_5mm.nii.gz'))
    hr_resample = Data_processing.resample_nifti(img_5mm, order=1,  mode = 'nearest',  cval = np.min(img_5mm.get_fdata()), in_plane_resolution_mm=new_dim[0], slice_thickness_mm=new_dim[-1])
    nb.save(hr_resample, os.path.join(save_folder, 'img_thinslice.nii.gz'))

    ######### select middle 100 slices
    img_thinslice = nb.load(os.path.join(save_folder, 'img_thinslice.nii.gz'))
    img_thinslice_data = img_thinslice.get_fdata()[:,:,img_thinslice.shape[2]//2-50:img_thinslice.shape[2]//2+50]
    nb.save(nb.Nifti1Image(img_thinslice_data, img_thinslice.affine, img_thinslice.header), os.path.join(save_folder, 'img_thinslice_partial.nii.gz'))


### step 2: insert two types of noise
- type 1: Poisson noise + hann filter
- type 2: Gaussian noise + soft tissue kernel

In [2]:
import numpy as np
import nibabel as nb 
import pandas as pd
import os
import copy
import CTDenoising_Diffusion_N2N.simulation.ct_basic_PCD as ct
import CTDenoising_Diffusion_N2N.functions_collection as ff
import ct_projector.projector.numpy.parallel as ct_para

main_path = '/mnt/camca_NAS/denoising/'

In [3]:
# define the patient list
patient_sheet = pd.read_excel(os.path.join(main_path,'Patient_lists', 'fixedCT_static.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
print('patient sheet len: ', len(patient_sheet))


simulation_num = 2
# do noise insertion, noise type = 'possion' (type1) or 'gaussian'(type2)
for i in range(0,len(patient_sheet)):
    row = patient_sheet.iloc[i]
    patient_id = row['Patient_ID']
    patient_subID = row['Patient_subID']

    print(patient_id, patient_subID)

    save_folder_case = os.path.join(main_path,'Data','simulation', patient_id,patient_subID)
    ff.make_folder([os.path.join(main_path,'Data','simulation'), os.path.join(main_path,'Data','simulation', patient_id), os.path.join(main_path,'Data','simulation', patient_id, patient_subID)])


    # noise-free thin-slice img file
    img_file = os.path.join(main_path,'Data/fixedCT',patient_id,patient_subID,'img_thinslice_partial.nii.gz')
    # load img
    img_clean = nb.load(img_file).get_fdata().astype(np.float32)
    img_clean[img_clean < -1024] = -1024
    spacing = nb.load(img_file).header.get_zooms()[::-1]
    affine = nb.load(img_file).affine

    for noise_type in ['possion', 'gaussian']: # two types
        #  we decided the following dose range by default
        possion_hann_dose_range = [0.10,0.20]
        gaussian_custom_dose_range = [0.15,0.25]
        
        for k in range(0,simulation_num):
            save_folder_k = os.path.join(save_folder_case, noise_type+'_random_'+str(k));ff.make_folder([save_folder_k])
            if os.path.isfile(os.path.join(save_folder_k,'recon.nii.gz')):
                print('already done, continue')
                continue

            if noise_type == 'possion':
                dose_factor = np.random.uniform(possion_hann_dose_range[0],possion_hann_dose_range[1] + 1e-8)
            elif noise_type == 'gaussian':
                dose_factor = np.random.uniform(gaussian_custom_dose_range[0],gaussian_custom_dose_range[1] + 1e-8)
            print('dose factor: ', dose_factor)

            # process img
            img0 = img_clean.copy()
            img0 = np.rollaxis(img0,-1,0)
            print('img shape, min, max: ', img0.shape, np.min(img0), np.max(img0))
            print('spacing: ', spacing)
      
            # define projectors
            projector = ct.define_forward_projector_pcd(img0,spacing, file_name = './help_data/pcd_parallel_6x5_512.cfg')

            # set angles
            angles = projector.get_angles()

            # recon
            recon_noise = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
            for slice_n in range(0, img0.shape[0]):
                img_slice = img0[[slice_n],:,:].copy()
                img_slice = (img_slice[np.newaxis, ...] + 1000) / 1000 * 0.019 

                prjs = ct_para.distance_driven_fp(projector, img_slice, angles)
                fprjs = ct_para.ramp_filter(projector, prjs, 'rl')

                # add noise
                if noise_type[0:2] == 'po':
                    # add poisson noise
                    noise_of_prjs = ct.add_poisson_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs
                elif noise_type[0:2] == 'ga':
                    # add gaussian noise
                    noise_of_prjs = ct.add_gaussian_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs

                # recon
                if noise_type[0:2] == 'po':
                    # hann filter
                    fnoise = ct_para.ramp_filter(projector, noise_of_prjs, 'hann')
                    recon_hann = ct_para.distance_driven_bp(projector, fnoise, angles, True) + img_slice
                    recon_hann = recon_hann[0, 0] / 0.019 * 1000 - 1000
                    recon_noise[:,:,slice_n] = recon_hann
                
                elif noise_type[0:2] == 'ga':
                    # custom filter
                    soft_tissue_kernel_file = './help_data/softTissueKernel_65'
                    custom_additional_filter = ct.get_additional_filter_to_rl(soft_tissue_kernel_file, projector.nu, projector.du, projector.nview)
                    recon_custom = ct.interleave_filter_and_recon(projector, noise_of_prjs, custom_additional_filter,angles) + img_slice
                    recon_custom = recon_custom[0, 0] / 0.019 * 1000 - 1000
                    recon_noise[:,:,slice_n] = recon_custom
            
            # save recon
            nb.save(nb.Nifti1Image(recon_noise, affine), os.path.join(save_folder_k,'recon.nii.gz'))

            

patient sheet len:  100
00214938 0000455723
dose factor:  0.15158064184329842
img shape, min, max:  (100, 512, 512) -1023.99994 1515.9093
spacing:  (0.625, 0.48046875, 0.48046875)
dose factor:  0.10827024880920996
img shape, min, max:  (100, 512, 512) -1023.99994 1515.9093
spacing:  (0.625, 0.48046875, 0.48046875)
dose factor:  0.22180748949719348
img shape, min, max:  (100, 512, 512) -1023.99994 1515.9093
spacing:  (0.625, 0.48046875, 0.48046875)


KeyboardInterrupt: 